## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [31]:
import numpy as np
import os
from random import shuffle
import re
import collections

In [32]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [33]:
import urllib.request
import zipfile
import lxml.etree

In [34]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [35]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [36]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [37]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [38]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [39]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [40]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [41]:
len(sentences_ted)

266694

In [42]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [43]:
counts_ted = collections.Counter()
tokens_ted = [token for sentence in sentences_ted for token in sentence]

for token in tokens_ted:
    counts_ted[token] += 1

In [44]:
counts_ted_top1000 = counts_ted.most_common(1000)

Plot distribution of top-1000 words

In [45]:
hist, edges = np.histogram([x[1] for x in counts_ted_top1000], density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [46]:
from gensim.models import Word2Vec

In [47]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=10, workers=1)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [48]:
model_ted.most_similar("man")

[('woman', 0.8323485851287842),
 ('guy', 0.8244491219520569),
 ('lady', 0.7795435190200806),
 ('boy', 0.763978123664856),
 ('girl', 0.7574023008346558),
 ('gentleman', 0.7141171097755432),
 ('soldier', 0.6908928155899048),
 ('poet', 0.6786893606185913),
 ('kid', 0.6668668389320374),
 ('rabbi', 0.659658670425415)]

In [49]:
model_ted.most_similar("computer")

[('machine', 0.7349832057952881),
 ('software', 0.6983097791671753),
 ('robot', 0.6935969591140747),
 ('device', 0.6575433015823364),
 ('3d', 0.6458271741867065),
 ('program', 0.6306606531143188),
 ('video', 0.6246740818023682),
 ('mechanical', 0.6219532489776611),
 ('camera', 0.614858865737915),
 ('mouse', 0.610051691532135)]

In [50]:
model_ted.most_similar("water")

[('air', 0.8449562788009644),
 ('heat', 0.8034026622772217),
 ('gas', 0.7626512050628662),
 ('food', 0.7375843524932861),
 ('soil', 0.7347041368484497),
 ('wind', 0.7335248589515686),
 ('fish', 0.7300703525543213),
 ('oil', 0.7274936437606812),
 ('sunlight', 0.7248802185058594),
 ('fresh', 0.7248294949531555)]

In [51]:
model_ted.most_similar("phone")

[('card', 0.7170721292495728),
 ('phones', 0.6993678212165833),
 ('facebook', 0.6678334474563599),
 ('mobile', 0.6541895270347595),
 ('file', 0.6432135105133057),
 ('iphone', 0.631677508354187),
 ('telephone', 0.6288535594940186),
 ('camera', 0.6208745241165161),
 ('cell', 0.6188596487045288),
 ('license', 0.6004078388214111)]

In [52]:
model_ted.most_similar("money")

[('attention', 0.6325304508209229),
 ('cash', 0.5851557850837708),
 ('food', 0.5767323970794678),
 ('credit', 0.5657031536102295),
 ('dollars', 0.5645880103111267),
 ('electricity', 0.5620164275169373),
 ('effort', 0.5595004558563232),
 ('funds', 0.5556673407554626),
 ('jobs', 0.5465083718299866),
 ('weight', 0.5413886308670044)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [53]:
words_top_ted = [pair[0] for pair in counts_ted_top1000]

# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [54]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [55]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [112]:
# Hand-In: an interesting cluster in the top left corner of  the T-SNE consists of units: i.e. miles, month, years, feet, week

### Part 5: Wiki Learnt Representations

Download dataset

In [85]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [98]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [99]:
sentences_strings_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_strings_wiki.extend(s)
    
for s_i in range(len(sentences_strings_wiki)):
    sentences_strings_wiki[s_i] = re.sub("[^a-z]", " ", sentences_strings_wiki[s_i].lower())
    sentences_strings_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_strings_wiki[s_i])
del input_text

In [100]:
# sample 1/5 of the data
shuffle(sentences_strings_wiki)
print(len(sentences_strings_wiki))
sentences_strings_wiki = sentences_strings_wiki[:int(len(sentences_strings_wiki)/5)]
print(len(sentences_strings_wiki))

4267112
853422


In [101]:
sentences_wiki = []
for sent_str in sentences_strings_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki.append(tokens)

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [102]:
counts_wiki = collections.Counter()
tokens_wiki = [token for sentence in sentences_wiki for token in sentence]

for token in tokens_wiki:
    counts_wiki[token] += 1
    
counts_wiki_top1000 = counts_wiki.most_common(1000)

hist, edges = np.histogram([x[1] for x in counts_wiki_top1000], density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [103]:
model_wiki = Word2Vec(sentences_wiki, size=100, window=5, min_count=10, workers=1)

In [104]:
model_wiki.most_similar("man")

[('woman', 0.6954430341720581),
 ('girl', 0.6509484648704529),
 ('boy', 0.6016116738319397),
 ('person', 0.5893335938453674),
 ('dog', 0.5880861282348633),
 ('mask', 0.5850304365158081),
 ('soldier', 0.5638561844825745),
 ('kid', 0.5495645999908447),
 ('creature', 0.547523021697998),
 ('men', 0.5333366990089417)]

In [105]:
model_wiki.most_similar("computer")

[('software', 0.8049265742301941),
 ('cgi', 0.7623907327651978),
 ('hardware', 0.761694073677063),
 ('computers', 0.7339255809783936),
 ('simulation', 0.721693754196167),
 ('computing', 0.7167501449584961),
 ('interface', 0.7002394795417786),
 ('controller', 0.69896399974823),
 ('interactive', 0.6970891952514648),
 ('gps', 0.6895076036453247)]

In [106]:
model_wiki.most_similar("water")

[('sediment', 0.7276384234428406),
 ('seawater', 0.6972014904022217),
 ('soil', 0.6954233646392822),
 ('stream', 0.6868279576301575),
 ('peat', 0.6830620765686035),
 ('drain', 0.6697967052459717),
 ('salt', 0.6679208874702454),
 ('oxygen', 0.6665977835655212),
 ('dry', 0.6652068495750427),
 ('marshes', 0.6618791818618774)]

In [107]:
model_wiki.most_similar("phone")

[('telephone', 0.6932331919670105),
 ('fake', 0.6031588315963745),
 ('phones', 0.5974645018577576),
 ('mulder', 0.5884917974472046),
 ('prank', 0.586082935333252),
 ('file', 0.5787581205368042),
 ('twitter', 0.557492733001709),
 ('laptop', 0.5561655163764954),
 ('email', 0.5556689500808716),
 ('youtube', 0.5497740507125854)]

In [108]:
model_wiki.most_similar("money")

[('cash', 0.7150408625602722),
 ('funds', 0.6921606063842773),
 ('payment', 0.670091986656189),
 ('compensation', 0.661243736743927),
 ('profits', 0.6422410011291504),
 ('debt', 0.632174015045166),
 ('loans', 0.6252145767211914),
 ('expenses', 0.6171555519104004),
 ('rent', 0.6132705211639404),
 ('revenue', 0.6083551645278931)]

#### t-SNE visualization

In [109]:
words_top_wiki = [pair[0] for pair in counts_wiki_top1000]

# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [110]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)